# Fáza 2 - Predspracovanie údajov

__Autori:__ Dávid Penťa, Samuel Bernát
__Percentuálny podiel práce:__ 50% / 50%

V tejto fáze sa od Vás očakáva že realizujte predspracovanie údajov pre strojové učenie. Výsledkom bude upravená dátová sada (csv alebo tsv), kde jedno pozorovanie je opísané jedným riadkom.
- scikit-learn vie len numerické dáta, takže treba niečo spraviť s nenumerickými dátami.
- Replikovateľnosť predspracovania na trénovacej a testovacej množine dát, aby ste mohli
zopakovať predspracovanie viackrát podľa Vašej potreby (iteratívne).

Keď sa predspracovaním mohol zmeniť tvar a charakteristiky dát, je možné že treba realizovať EDA opakovane podľa Vašej potreby. Bodovanie znovu (EDA) nebudeme, zmeny ale dokumentujte. Problém s dátami môžete riešiť iteratívne v každej fáze aj vo všetkých fázach podľa potreby.

In [68]:
import statsmodels.api as sm
import dateparser as dateparser
import matplotlib
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scipy.stats as stats
import pylab as py
import statsmodels.stats.api as sms
from scipy.stats import skew
from scipy.stats import kurtosis
from scipy.stats import pearsonr
import math
from matplotlib import pyplot
from numpy import exp
from numpy.random import randn
from sklearn.preprocessing import PowerTransformer
from operator import itemgetter
from sklearn.impute import KNNImputer

In [93]:
measurements_file = "data/measurements.csv"
measurements_data = pd.read_csv(measurements_file, sep='\t')
measurements_data_med = pd.read_csv(measurements_file, sep='\t')
measurements_data_avg = pd.read_csv(measurements_file, sep='\t')
measurements_data_knn = pd.read_csv(measurements_file, sep='\t')
measurements_data_out = pd.read_csv(measurements_file, sep='\t')

stations_file = "data/stations.csv"
stations_data = pd.read_csv(stations_file, sep='\t')

## Integrácia a čistenie dát
Transformujte dáta na vhodný formát pre strojové učenie t.j. jedno pozorovanie musí byť opísané jedným riadkom a každý atribút musí byť v numerickom formáte.
- Pri riešení chýbajúcich hodnôt (missing values) vyskúšajte rôzne stratégie ako napr.
    - odstránenie pozorovaní s chýbajúcimi údajmi
    - nahradenie chýbajúcej hodnoty mediánom, priemerom, pomerom (ku korelovanému atribútu), alebo pomocou lineárnej regresie resp. kNN
- Podobne postupujte aj pri riešení vychýlených hodnôt (outlier detection):
    - odstránenie vychýlených (odľahlých) pozorovaní
    - nahradenie vychýlenej hodnoty hraničnými hodnotami rozdelenia (5% resp. 95%)

### 1. Chýbajúce hodnoty
Prvou použitou technikou odstránim pozorovania s chýbajúcimi údajmi (v tvare Nan).

In [70]:
measurements_data.dropna(inplace=True)
stations_data.dropna(inplace=True)

stations_data["QoS"] = np.where(stations_data["QoS"] == "accep", "acceptable", stations_data["QoS"])
stations_data["QoS"] = np.where(stations_data["QoS"] == "maitennce", "maintenance", stations_data["QoS"])

# stations_data['revision'] = stations_data['revision'].apply(lambda x: pd.Timestamp(x).strftime('%B-%d-%Y'))
# stations_data['revision_timestamp'] = stations_data['revision'].apply(lambda x: pd.Timestamp(x).timestamp())
stations_data['revision'] = stations_data['revision'].apply(lambda x: pd.Timestamp(x).timestamp())

stations_data['latitude'] = stations_data['latitude'].round(5)
stations_data['longitude'] = stations_data['longitude'].round(5)

stations_data["station"] = np.where(stations_data["station"] == "T‚Äôaebaek", "Taebaek", stations_data["station"])
stations_data["station"] = np.where(stations_data["station"] == "'Ali Sabieh", "Ali Sabieh", stations_data["station"])
stations_data["station"] = np.where(stations_data["station"] == "Oktyabr‚Äôskiy", "Oktyabrsk", stations_data["station"])
stations_data["station"] = np.where(stations_data["station"] == "Roslavl‚Äô", "Roslavl", stations_data["station"])
stations_data["station"] = np.where(stations_data["station"] == "Dyat‚Äôkovo", "Dyatkovo", stations_data["station"])

#stations_data
measurements_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11354 entries, 0 to 12067
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   PM10       11354 non-null  float64
 1   CO         11354 non-null  float64
 2   Pb         11354 non-null  float64
 3   C2H3NO5    11354 non-null  float64
 4   CFCs       11354 non-null  float64
 5   H2CO       11354 non-null  float64
 6   O3         11354 non-null  float64
 7   TEMP       11354 non-null  float64
 8   NOx        11354 non-null  float64
 9   SO2        11354 non-null  float64
 10  latitude   11354 non-null  float64
 11  longitude  11354 non-null  float64
 12  NH3        11354 non-null  float64
 13  CH4        11354 non-null  float64
 14  PRES       11354 non-null  float64
 15  PM2.5      11354 non-null  float64
 16  warning    11354 non-null  float64
 17  PAHs       11354 non-null  float64
dtypes: float64(18)
memory usage: 1.6 MB


Druhou technikou nahradím v datasete chýbajúce hodnoty mediánom daného stĺpca.

In [49]:
for col in measurements_data_med:
    measurements_data_med[col].fillna((measurements_data[col].median()), inplace=True)

Treťou technikou nahradím v datasete chýbajúce hodnoty priemerom daného stĺpca.

In [50]:
for col in measurements_data_avg:
    measurements_data_avg[col].fillna((measurements_data[col].mean()), inplace=True)

Poslenou technikou nahradím v datasete chýbajúce hodnoty hodnotou vypočítanou metódou kNN, s nastavením na 5 susedov.

In [51]:
data = measurements_data_knn.values
ix = [i for i in range(data.shape[1]) if i != 17]
X, y = data[:, ix], data[:, 17]

imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
imputer.fit(X)
Xtrans = imputer.transform(X)

### 1. Outlier detection
Použitím prvej techniky odstránim vychýlené pozorovania z datasetu.

In [94]:
print("Old: ", measurements_data.shape)
for col in measurements_data_out:
    Q1 = np.percentile(measurements_data_out[col], 25, method = 'midpoint')
    Q3 = np.percentile(measurements_data_out[col], 75, method = 'midpoint')
    IQR = Q3 - Q1

    upper = np.where(measurements_data_out[col] >= (Q3+1.5*IQR))
    lower = np.where(measurements_data_out[col] <= (Q1-1.5*IQR))

for i in range(len(upper)):
    measurements_data_out.drop(upper[i], inplace = True)
for j in range(len(lower)):
    measurements_data_out.drop(lower[i], inplace = True)

print("New: ", measurements_data.shape)

Old:  (12068, 18)
New:  (12068, 18)


In [90]:
import sklearn
from sklearn.datasets import load_boston
import pandas as pd

# Load the dataset
bos_hou = load_boston()

# Create the dataframe
column_name = bos_hou.feature_names
df_boston = pd.DataFrame(bos_hou.data)
df_boston.columns = column_name
df_boston.head()

''' Detection '''
# IQR
Q1 = np.percentile(df_boston['DIS'], 25,
                   interpolation = 'midpoint')

Q3 = np.percentile(df_boston['DIS'], 75,
                   interpolation = 'midpoint')
IQR = Q3 - Q1

print("Old Shape: ", df_boston.shape)

# Upper bound
upper = np.where(df_boston['DIS'] >= (Q3+1.5*IQR))
# Lower bound
lower = np.where(df_boston['DIS'] <= (Q1-1.5*IQR))
print(lower, upper)

''' Removing the Outliers '''
df_boston.drop(upper[0], inplace = True)
df_boston.drop(lower[0], inplace = True)

print("New Shape: ", df_boston.shape)

Old Shape:  (506, 13)
(array([], dtype=int64),) (array([351, 352, 353, 354, 355], dtype=int64),)
New Shape:  (501, 13)


C:\Users\Samko\Documents\GitHub\IAU\venv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California ho

In [ ]:
stations_data = stations_data.groupby(by='station').apply(lambda x: x.loc[x['revision']==x['revision'].max()])

## Spojenie tabuliek

In [ ]:
df=pd.merge(stations_data, measurements_data, on=['latitude', 'longitude'], how='inner')

df

### Opis každého atribútu numerickým formátom

In [ ]:
df['QoS_ID'] = -1

df.loc[df.QoS == "excellent", "QoS_ID"] = "1"
df.loc[df.QoS == "good", "QoS_ID"] = "2"
df.loc[df.QoS == "average", "QoS_ID"] = "3"
df.loc[df.QoS == "acceptable", "QoS_ID"] = "4"
df.loc[df.QoS == "building", "QoS_ID"] = "5"
df.loc[df.QoS == "maintenance", "QoS_ID"] = "5"

df[['QoS_ID']] = df[['QoS_ID']].apply(pd.to_numeric)

In [ ]:
df['station_ID'] = df['station'].rank(method='dense')
df['code_ID'] = df['code'].rank(method='dense')

df

### Uloženie upravenej dátovej sady do .csv súboru

In [ ]:
output = df[['station_ID', 'code_ID', 'QoS_ID', 'warning', 'latitude', 'longitude', 'revision','PAHs', 'PM10', 'CO', 'Pb', 'C2H3NO5', 'CFCs', 'H2CO', 'O3', 'TEMP', 'NOx', 'SO2', 'NH3', 'CH4', 'PRES', 'PM2.5']]

output.to_csv('output.csv', index=False)

### Power Transformer

In [ ]:
def transform(column_name):

    sns.histplot(data=df, hue='warning', x=column_name, fill=True, kde=True)
    plt.show()

    data = df[column_name].values
    data = data.reshape((len(data),1))

    power = PowerTransformer(
                method='yeo-johnson',
                standardize=True)
    data_trans = power.fit_transform(data)
    df[column_name] = data_trans

    sns.histplot(data=df, hue='warning', x=column_name, fill=True, kde=True)
    plt.show()

In [ ]:
transform('C2H3NO5')

# arr_0 = ['PAHs', 'PM10', 'CO', 'Pb', 'C2H3NO5', 'CFCs', 'H2CO', 'O3', 'TEMP', 'NOx', 'SO2', 'NH3', 'CH4', 'PRES', 'PM2.5']
# for i in arr_0:
#     print(i)
#     transform(i)

### Zoradenie podla p a zaroven r

In [ ]:
arr = []
#
for i in list(df.columns.values):
    if i != 'QoS' and i != 'station' and i != 'code':
        (r, p) = pearsonr(df['warning'], df[i])
        if p < 0.05 and i != 'warning':
            arr.append([i, abs(r), p])

arr = sorted(arr, key=itemgetter(1))
a = []
for i in range(len(arr)):
    a.append([i + 1, arr[len(arr) - i - 1][0], arr[len(arr) - i - 1][1], arr[len(arr) - i - 1][2]])

for row in a:
    print("{: >0} {: >10} {:.5f} {:.5f}".format(*row))